## Setup & Configuration <a id="setup"></a>

First, let's load environment variables and import common dependencies.

In [1]:
# Copyright (c) Microsoft. All rights reserved.

import asyncio
import os
import httpx
from typing import Annotated
from pathlib import Path
from dotenv import load_dotenv
from pydantic import Field

# Load environment variables
local_env_path = Path.cwd() / ".env"
parent_env_path = Path.cwd().parent.parent.parent / ".env"
load_dotenv(dotenv_path=local_env_path)
load_dotenv(dotenv_path=parent_env_path)

print("Environment loaded successfully!")
print(f"Project Endpoint: {os.getenv('AZURE_AI_PROJECT_ENDPOINT', 'Not set')}")
print(f"Model Deployment: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME', 'Not set')}")

Environment loaded successfully!
Project Endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject
Model Deployment: gpt-4.1


### Helper Functions

Define reusable helper functions for weather data.

In [2]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the current weather for a given location using OpenWeatherMap API."""
    api_key = os.getenv("OPENWEATHER_API_KEY")
    if not api_key:
        return f"Error: OPENWEATHER_API_KEY not found in environment variables."
    
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_key}&units=metric"
        response = httpx.get(url, timeout=10.0)
        response.raise_for_status()
        data = response.json()
        
        temp = data["main"]["temp"]
        feels_like = data["main"]["feels_like"]
        description = data["weather"][0]["description"]
        humidity = data["main"]["humidity"]
        
        return f"The weather in {location} is {description} with a temperature of {temp}°C (feels like {feels_like}°C) and {humidity}% humidity."
    except httpx.HTTPStatusError as e:
        if e.response.status_code == 404:
            return f"Error: Location '{location}' not found."
        return f"Error fetching weather data: {e}"
    except Exception as e:
        return f"Error: {str(e)}"

print("Helper functions defined!")

Helper functions defined!


---

## 1. Basic Agent <a id="basic"></a>

The simplest way to create an agent using `AzureAIClient`. Demonstrates both streaming and non-streaming responses.

In [3]:
from agent_framework.azure import AzureAIClient
from azure.identity.aio import AzureCliCredential

async def basic_agent_demo():
    print("=== Basic Azure AI Agent Example ===")
    
    # Non-streaming example
    print("\n--- Non-streaming Response ---")
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="BasicWeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        query = "What's the weather like in Toronto?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")
    
    # Streaming example
    print("\n--- Streaming Response ---")
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="BasicWeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        query = "What's the weather like in Mexico City?"
        print(f"User: {query}")
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print()

await basic_agent_demo()

=== Basic Azure AI Agent Example ===

--- Non-streaming Response ---
User: What's the weather like in Toronto?
Agent: Currently in Toronto, there are a few clouds with a temperature of 7.3°C (feels like 5°C) and humidity is at 66%. It's a cool, slightly cloudy day.

--- Streaming Response ---
User: What's the weather like in Mexico City?
Agent: The current weather in Mexico City is clear with a temperature of 22.75°C (feels like 21.6°C) and 20% humidity.


---

## 2. Using Latest Version <a id="latest-version"></a>

Demonstrates how to reuse the latest version of an existing agent instead of creating a new agent version on each instantiation.

In [4]:
async def latest_version_demo():
    print("=== Using Latest Agent Version ===")
    
    async with AzureCliCredential() as credential:
        # First call creates a new agent
        async with (
            AzureAIClient(
                async_credential=credential,
            ).create_agent(
                name="MyWeatherAgent",
                instructions="You are a helpful weather agent.",
                tools=get_weather,
            ) as agent,
        ):
            query = "What's the weather like in Toronto?"
            print(f"User: {query}")
            result = await agent.run(query)
            print(f"Agent: {result}")

        # Second call reuses latest version
        print("\n--- Reusing Latest Version ---")
        async with (
            AzureAIClient(
                async_credential=credential,
                use_latest_version=True,  # This parameter reuses the latest agent version
            ).create_agent(
                name="MyWeatherAgent",
                instructions="You are a helpful weather agent.",
                tools=get_weather,
            ) as agent,
        ):
            query = "What's the weather like in Mexico City?"
            print(f"User: {query}")
            result = await agent.run(query)
            print(f"Agent: {result}")

await latest_version_demo()

=== Using Latest Agent Version ===
User: What's the weather like in Toronto?
Agent: The weather in Toronto is currently partly cloudy with a temperature of about 7.3°C (feels like 5°C) and 66% humidity.

--- Reusing Latest Version ---
User: What's the weather like in Mexico City?
Agent: The weather in Mexico City is clear with a temperature of 22.75°C (feels like 21.6°C) and 20% humidity. It's a pleasant day with clear skies!


---

## 3. Agent-to-Agent (A2A) <a id="a2a"></a>

Demonstrates Agent-to-Agent communication using the A2A protocol.

**Prerequisites:** Requires `A2A_PROJECT_CONNECTION_ID` environment variable.

---

## 4. Azure AI Search <a id="azure-search"></a>

Search through indexed data using Azure AI Search.

**Prerequisites:** `AI_SEARCH_PROJECT_CONNECTION_ID` and `AI_SEARCH_INDEX_NAME` environment variables.

In [5]:
async def azure_search_demo():
    print("=== Azure AI Search Example ===")
    
    if not os.getenv("AI_SEARCH_PROJECT_CONNECTION_ID") or not os.getenv("AI_SEARCH_INDEX_NAME"):
        print("⚠️  Skipping: AI_SEARCH_PROJECT_CONNECTION_ID and AI_SEARCH_INDEX_NAME required")
        return
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MySearchAgent",
            instructions="""You are a helpful assistant. You must always provide citations for
            answers using the tool and render them as: `[message_idx:search_idx†source]`.""",
            tools={
                "type": "azure_ai_search",
                "azure_ai_search": {
                    "indexes": [
                        {
                            "project_connection_id": os.environ["AI_SEARCH_PROJECT_CONNECTION_ID"],
                            "index_name": os.environ["AI_SEARCH_INDEX_NAME"],
                            "query_type": "simple",
                        }
                    ]
                },
            },
        ) as agent,
    ):
        query = "Tell me about what cities I can visit"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await azure_search_demo()

=== Azure AI Search Example ===
User: Tell me about what cities I can visit
Agent: Sure! There are countless cities around the world you can visit, each offering unique experiences, culture, and landmarks. Here are a few popular options grouped by region:

**North America**
- **New York City, USA:** Famous for Times Square, Central Park, Broadway shows, and diverse neighborhoods.
- **Toronto, Canada:** Known for its multicultural vibe, CN Tower, and waterfront.
- **Mexico City, Mexico:** A bustling metropolis with ancient ruins, vibrant street food, and rich history.

**Europe**
- **Paris, France:** Renowned for the Eiffel Tower, art museums, cafes, and romantic atmosphere.
- **London, UK:** Iconic sights include Buckingham Palace, Big Ben, and world-class museums.
- **Rome, Italy:** Home to ancient ruins like the Colosseum and Vatican City.

**Asia**
- **Tokyo, Japan:** Combines modern skyscrapers with historic temples, cherry blossoms, and culinary delights.
- **Bangkok, Thailand:** 

---

## 5. Bing Custom Search <a id="bing-custom"></a>

Search custom search instances using Bing Custom Search.

**Prerequisites:** `BING_CUSTOM_SEARCH_PROJECT_CONNECTION_ID` and `BING_CUSTOM_SEARCH_INSTANCE_NAME`.

---

## 6. Bing Grounding <a id="bing-grounding"></a>

Search the web for current information using Bing Grounding.

**Prerequisites:** `BING_PROJECT_CONNECTION_ID` environment variable.

In [6]:
async def bing_grounding_demo():
    print("=== Bing Grounding Example ===")
    
    if not os.getenv("BING_PROJECT_CONNECTION_ID"):
        print("⚠️  Skipping: BING_PROJECT_CONNECTION_ID required")
        return
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MyBingGroundingAgent",
            instructions="""You are a helpful assistant that can search the web for current information.
            Use the Bing search tool to find up-to-date information and provide accurate, well-sourced answers.
            Always cite your sources when possible.""",
            tools={
                "type": "bing_grounding",
                "bing_grounding": {
                    "search_configurations": [
                        {
                            "project_connection_id": os.environ["BING_PROJECT_CONNECTION_ID"],
                        }
                    ]
                },
            },
        ) as agent,
    ):
        query = "What is today's date and weather in Toronto?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await bing_grounding_demo()

=== Bing Grounding Example ===
User: What is today's date and weather in Toronto?
Agent: Today's date is November 24, 2025.

As for the weather in Toronto today:
- The temperature is ranging between 0°C and 8°C.
- It is mostly overcast, with periods of patchy rain possible and a high probability of precipitation (up to 82% chance), though accumulations are light.
- Winds are breezy, averaging around 24 km/h.
- Current conditions around the morning hours (7-9am) report temperatures of about 4-6°C and a "feels like" temperature of 1-2°C.

So, in summary: It is a cool, breezy, and mainly overcast day with light rain possible in Toronto on November 24, 2025.


---

## 7. Code Interpreter <a id="code-interpreter"></a>

Execute Python code for mathematical problem solving using the `HostedCodeInterpreterTool`.

In [7]:
from agent_framework import ChatResponse, HostedCodeInterpreterTool
from openai.types.responses.response import Response as OpenAIResponse
from openai.types.responses.response_code_interpreter_tool_call import ResponseCodeInterpreterToolCall

async def code_interpreter_demo():
    print("=== Code Interpreter Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MyCodeInterpreterAgent",
            instructions="You are a helpful assistant that can write and execute Python code to solve problems.",
            tools=HostedCodeInterpreterTool(),
        ) as agent,
    ):
        query = "Use code to get the factorial of 100?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

        # Extract generated code if available
        if (
            isinstance(result.raw_representation, ChatResponse)
            and isinstance(result.raw_representation.raw_representation, OpenAIResponse)
            and len(result.raw_representation.raw_representation.output) > 0
        ):
            code_interpreter_item = next(
                (
                    item
                    for item in result.raw_representation.raw_representation.output
                    if isinstance(item, ResponseCodeInterpreterToolCall)
                ),
                None,
            )

            if code_interpreter_item is not None:
                generated_code = code_interpreter_item.code
                print(f"\nGenerated code:\n{generated_code}")

await code_interpreter_demo()

=== Code Interpreter Example ===
User: Use code to get the factorial of 100?
Agent: # Calculate the factorial of 100
import math

factorial_100 = math.factorial(100)
factorial_100 The factorial of 100 is:

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

If you need it saved to a file or formatted differently, let me know!

Generated code:
# Calculate the factorial of 100
import math

factorial_100 = math.factorial(100)
factorial_100


---

## 8. Existing Agent <a id="existing-agent"></a>

Work with a pre-existing agent by providing agent name and version.

In [8]:
from agent_framework import ChatAgent
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition

async def existing_agent_demo():
    print("=== Existing Agent Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as project_client,
    ):
        # Create a test agent
        azure_ai_agent = await project_client.agents.create_version(
            agent_name="MyNewTestAgent",
            definition=PromptAgentDefinition(
                model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
                instructions="End each response with [END].",
            ),
        )

        chat_client = AzureAIClient(
            project_client=project_client,
            agent_name=azure_ai_agent.name,
            agent_version=azure_ai_agent.version,
        )

        try:
            async with ChatAgent(
                chat_client=chat_client,
            ) as agent:
                query = "How are you?"
                print(f"User: {query}")
                result = await agent.run(query)
                print(f"Agent: {result}")
        finally:
            # Cleanup
            await project_client.agents.delete_version(
                agent_name=azure_ai_agent.name, agent_version=azure_ai_agent.version
            )

await existing_agent_demo()

=== Existing Agent Example ===
User: How are you?
Agent: I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you! How can I assist you today? [END]


---

## 9. Existing Conversation <a id="existing-conversation"></a>

Use an existing conversation created on the service side.

In [9]:
async def existing_conversation_demo():
    print("=== Existing Conversation Example ===")
    print("Note: This example demonstrates thread management with service-side storage.")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="BasicAgent",
            instructions="You are a helpful agent.",
            tools=get_weather,
        ) as agent,
    ):
        # Create a new thread (which creates a conversation on the service)
        thread = agent.get_new_thread()

        query = "What's the weather like in Guadalajara?"
        print(f"User: {query}")
        result = await agent.run(query, thread=thread)
        print(f"Agent: {result.text}")
        
        # The thread now has a service-side ID
        if thread.service_thread_id:
            print(f"Thread ID: {thread.service_thread_id}")

        query = "What was my last question?"
        print(f"\nUser: {query}")
        result = await agent.run(query, thread=thread)
        print(f"Agent: {result.text}")

await existing_conversation_demo()

=== Existing Conversation Example ===
Note: This example demonstrates thread management with service-side storage.
User: What's the weather like in Guadalajara?
Agent: The current weather in Guadalajara is partly cloudy, with a temperature of about 24.9°C and 36% humidity. It feels pleasantly warm.
Thread ID: resp_088bcf953d698b1f006924b11e08b8819483459999d15b9eba

User: What was my last question?
Agent: Your last question was: "What's the weather like in Guadalajara?"


---

## 10. Explicit Settings <a id="explicit-settings"></a>

Create an agent with explicitly configured settings instead of environment defaults.

In [10]:
from random import randint

def get_mock_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get mock weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

async def explicit_settings_demo():
    print("=== Explicit Settings Example ===")
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIClient(
                project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
                model_deployment_name=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
                async_credential=credential,
                agent_name="WeatherAgent",
            ),
            instructions="You are a helpful weather agent.",
            tools=get_mock_weather,
        ) as agent,
    ):
        query = "What's the weather like in New York?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await explicit_settings_demo()

=== Explicit Settings Example ===
User: What's the weather like in New York?
Agent: The weather in New York is currently cloudy with a high temperature of 12°C.


---

## 11. File Search <a id="file-search"></a>

Upload files and enable agents to search through documents using `HostedFileSearchTool`.

In [11]:
from agent_framework import HostedFileSearchTool, HostedVectorStoreContent
from azure.ai.agents.aio import AgentsClient
from azure.ai.agents.models import FileInfo, VectorStore

async def file_search_demo():
    print("=== File Search Example ===")
    
    file = None
    vector_store = None

    async with (
        AzureCliCredential() as credential,
        AgentsClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as agents_client,
        AzureAIClient(async_credential=credential) as client,
    ):
        try:
            # Upload file
            pdf_file_path = Path.cwd().parent / "resources" / "employees.pdf"
            if not pdf_file_path.exists():
                print(f"⚠️  Skipping: File not found at {pdf_file_path}")
                return
                
            print(f"Uploading file: {pdf_file_path}")
            file = await agents_client.files.upload_and_poll(file_path=str(pdf_file_path), purpose="assistants")
            print(f"Uploaded file ID: {file.id}")

            vector_store = await agents_client.vector_stores.create_and_poll(file_ids=[file.id], name="my_vectorstore")
            print(f"Created vector store ID: {vector_store.id}")

            # Create file search tool
            file_search_tool = HostedFileSearchTool(inputs=[HostedVectorStoreContent(vector_store_id=vector_store.id)])

            async with ChatAgent(
                chat_client=client,
                name="EmployeeSearchAgent",
                instructions="You are a helpful assistant that can search through uploaded employee files.",
                tools=file_search_tool,
            ) as agent:
                queries = [
                    "Who is the youngest employee?",
                    "Who works in sales?",
                ]
                for query in queries:
                    print(f"\nUser: {query}")
                    response = await agent.run(query)
                    print(f"Agent: {response.text}")
        finally:
            # Cleanup
            if vector_store:
                await agents_client.vector_stores.delete(vector_store.id)
            if file:
                await agents_client.files.delete(file.id)

await file_search_demo()

=== File Search Example ===
Uploading file: /Users/arturoquiroga/GITHUB/agent-framework-public/maf-upstream/python/samples/getting_started/agents/resources/employees.pdf
Uploaded file ID: assistant-2zY1aKs8KvbW4H4xf7Vqx6
Created vector store ID: vs_snmF6XzAqsTSg8ke5QZPI1ir

User: Who is the youngest employee?
Agent: The youngest employee is Alice Johnson, who is 24 years old.

User: Who works in sales?
Agent: Alice Johnson works in the Sales department.


---

## 12. Hosted MCP <a id="hosted-mcp"></a>

Integrate hosted Model Context Protocol (MCP) tools with Azure AI Agent.

In [12]:
from agent_framework import HostedMCPTool

async def hosted_mcp_demo():
    print("=== Hosted MCP Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="MyLearnDocsAgent",
            instructions="You are a helpful assistant that can help with Microsoft documentation questions.",
            tools=HostedMCPTool(
                name="Microsoft Learn MCP",
                url="https://learn.microsoft.com/api/mcp",
                approval_mode="never_require",
            ),
        ) as agent,
    ):
        query = "How to create an Azure storage account using az cli?"
        print(f"User: {query}")
        result = await agent.run(query, store=False)
        print(f"Agent: {result}")

await hosted_mcp_demo()

=== Hosted MCP Example ===
User: How to create an Azure storage account using az cli?
Agent: To **create an Azure storage account using Azure CLI (az cli)**, follow these steps:

### 1. **Log In to Azure**
```bash
az login
```

### 2. **Set the Subscription (optional)**
If you have more than one subscription:
```bash
az account set --subscription "<subscription-name-or-id>"
```

### 3. **Create a Resource Group**
```bash
az group create --name <resource-group-name> --location <location>
```

### 4. **Create the Storage Account**
```bash
az storage account create \
  --name <storage-account-name> \
  --resource-group <resource-group-name> \
  --location <location> \
  --sku Standard_LRS
```
**Parameters:**
- `--name`: Must be unique across Azure, 3-24 characters, lowercase only.
- `--resource-group`: The resource group you created above.
- `--location`: Azure region, e.g., `eastus`, `westeurope`, etc.
- `--sku`: Pricing tier, e.g., `Standard_LRS` (locally redundant storage).

### **Exam

---

## 13. Image Generation <a id="image-generation"></a>

Generate images based on text prompts using the `ImageGenTool`.

In [13]:
from agent_framework import DataContent
from azure.ai.projects.models import ImageGenTool
import aiofiles

async def image_generation_demo():
    print("=== Image Generation Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="ImageGenAgent",
            instructions="Generate images based on user requirements.",
            tools=[ImageGenTool(quality="low", size="1024x1024")],
        ) as agent,
    ):
        query = "Generate an image of Microsoft logo."
        print(f"User: {query}")
        result = await agent.run(
            query,
            additional_chat_options={
                "extra_headers": {"x-ms-oai-image-generation-deployment": "gpt-image-1"},
            },
        )
        print(f"Agent: {result}")

        # Save image if available
        image_data = [
            content
            for content in result.messages[0].contents
            if isinstance(content, DataContent) and content.media_type == "image/png"
        ]
        if image_data and image_data[0]:
            filename = "microsoft3.png"
            file_path = Path.cwd() / filename
            async with aiofiles.open(file_path, "wb") as f:
                await f.write(image_data[0].get_data_bytes())
            print(f"Image saved to: {file_path}")


await image_generation_demo()

[2025-11-24 14:25:59 - /Users/arturoquiroga/GITHUB/agent-framework-public/.venv/lib/python3.13/site-packages/agent_framework/_tools.py:827 - WARNING] Can't parse tool.
[2025-11-24 14:25:59 - /Users/arturoquiroga/GITHUB/agent-framework-public/.venv/lib/python3.13/site-packages/agent_framework/_tools.py:827 - WARNING] Can't parse tool.


=== Image Generation Example ===
User: Generate an image of Microsoft logo.
Agent: Here is an image inspired by the Microsoft logo, featuring four colored squares and the word "Microsoft" in modern font.
Image saved to: /Users/arturoquiroga/GITHUB/agent-framework-public/maf-upstream/python/samples/getting_started/agents/azure_ai/microsoft3.png


---

## 14. Response Format (Structured Outputs) <a id="response-format"></a>

Use Pydantic models to enforce specific response schemas.

In [14]:
from pydantic import BaseModel, ConfigDict

class ReleaseBrief(BaseModel):
    feature: str
    benefit: str
    launch_date: str
    model_config = ConfigDict(extra="forbid")

async def response_format_demo():
    print("=== Response Format (Structured Outputs) Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="ProductMarketerAgent",
            instructions="Return launch briefs as structured JSON.",
        ) as agent,
    ):
        query = "Draft a launch brief for the Contoso Note app."
        print(f"User: {query}")
        result = await agent.run(query, response_format=ReleaseBrief)

        if isinstance(result.value, ReleaseBrief):
            release_brief = result.value
            print("Agent:")
            print(f"Feature: {release_brief.feature}")
            print(f"Benefit: {release_brief.benefit}")
            print(f"Launch date: {release_brief.launch_date}")

await response_format_demo()

=== Response Format (Structured Outputs) Example ===
User: Draft a launch brief for the Contoso Note app.
Agent:
Feature: Contoso Note app
Benefit: Enables users to quickly capture, organize, and sync notes across devices, improving productivity and collaboration.
Launch date: 2024-06-25


---

## 17. Thread Management <a id="thread-management"></a>

Demonstrate thread persistence and conversation context management.

In [15]:
async def thread_management_demo():
    print("=== Thread Management Example ===")
    
    # Example with thread persistence in-memory
    print("\n--- Thread Persistence (In-Memory) ---")
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="BasicWeatherAgent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        ) as agent,
    ):
        thread = agent.get_new_thread()

        query1 = "What's the weather like in Mexico City?"
        print(f"User: {query1}")
        result1 = await agent.run(query1, thread=thread, store=False)
        print(f"Agent: {result1.text}")

        query2 = "How about Guadalajara?"
        print(f"\nUser: {query2}")
        result2 = await agent.run(query2, thread=thread, store=False)
        print(f"Agent: {result2.text}")

        query3 = "Which of the cities I asked about has better weather?"
        print(f"\nUser: {query3}")
        result3 = await agent.run(query3, thread=thread, store=False)
        print(f"Agent: {result3.text}")

await thread_management_demo()

=== Thread Management Example ===

--- Thread Persistence (In-Memory) ---
User: What's the weather like in Mexico City?
Agent: The weather in Mexico City is currently clear with a temperature of about 22.8°C, feeling slightly cooler at 21.6°C. Humidity is low at 20%. It looks like a pleasant, sunny day!

User: How about Guadalajara?
Agent: Guadalajara is experiencing scattered clouds today with a temperature of around 24.9°C, feeling like 24.4°C. The humidity is at 36%, making it a comfortable day overall.

User: Which of the cities I asked about has better weather?
Agent: Here’s a comparison of the current weather in Mexico City and Guadalajara:

- **Mexico City:** Clear sky, 22.8°C (feels like 21.6°C), 20% humidity.
- **Guadalajara:** Scattered clouds, 24.9°C (feels like 24.4°C), 36% humidity.

**Mexico City** has clearer skies and lower humidity, making for a crisp and sunny day. **Guadalajara** is a bit warmer with some clouds and higher humidity, but still pleasant.

If you prefer

---

## 18. Web Search <a id="web-search"></a>

Perform web searches using the `HostedWebSearchTool`.

In [16]:
from agent_framework import HostedWebSearchTool

async def web_search_demo():
    print("=== Web Search Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="WebsearchAgent",
            instructions="You are a helpful assistant that can search the web",
            tools=[HostedWebSearchTool()],
        ) as agent,
    ):
        query = "What's the weather today in Toronto?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await web_search_demo()

=== Web Search Example ===
User: What's the weather today in Toronto?
Agent: Today in Toronto, expect mostly sunny conditions with a high of about 6°C and a low near 4°C. Winds will be coming from the southeast at around 13 km/h, and the chance of precipitation is low. It will feel slightly colder due to the wind chill, especially in the morning. Sunrise was at 7:16 am and sunset will be at 4:49 pm, providing just over 9 hours of daylight[Weather today - Toronto, Canada](https://www.weather-atlas.com/en/canada/toronto).


In [17]:
from agent_framework import DataContent
from azure.ai.projects.models import ImageGenTool
import aiofiles
from pathlib import Path

async def image_generation_demo():
    print("=== Image Generation Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIClient(async_credential=credential).create_agent(
            name="ImageGenAgent",
            instructions="Generate images based on user requirements.",
            tools=[ImageGenTool(quality="low", size="1024x1024")],
        ) as agent,
    ):
        query = "Generate an image of Microsoft logo."
        print(f"User: {query}")
        result = await agent.run(
            query,
            additional_chat_options={
                "extra_headers": {"x-ms-oai-image-generation-deployment": "gpt-image-1"},
            },
        )
        print(f"Agent: {result}")

        # Save image if available
        image_data = [
            content
            for content in result.messages[0].contents
            if isinstance(content, DataContent) and content.media_type == "image/png"
        ]
        if image_data and image_data[0]:
            filename = "microsoft3.png"
            file_path = Path.cwd() / filename
            async with aiofiles.open(file_path, "wb") as f:
                await f.write(image_data[0].get_data_bytes())
            print(f"Image saved to: {file_path}")


await image_generation_demo()

[2025-11-24 14:26:51 - /Users/arturoquiroga/GITHUB/agent-framework-public/.venv/lib/python3.13/site-packages/agent_framework/_tools.py:827 - WARNING] Can't parse tool.
[2025-11-24 14:26:51 - /Users/arturoquiroga/GITHUB/agent-framework-public/.venv/lib/python3.13/site-packages/agent_framework/_tools.py:827 - WARNING] Can't parse tool.


=== Image Generation Example ===
User: Generate an image of Microsoft logo.
Agent: Here is an image of the Microsoft logo.
Image saved to: /Users/arturoquiroga/GITHUB/agent-framework-public/maf-upstream/python/samples/getting_started/agents/azure_ai/microsoft3.png


---

## Summary

This notebook demonstrated 18 different patterns for working with Azure AI agents:

1. ✅ Basic agent with streaming/non-streaming
2. ✅ Using latest agent version
3. ✅ Agent-to-Agent communication
4. ✅ Azure AI Search integration
5. ✅ Bing Custom Search
6. ✅ Bing Grounding for web search
7. ✅ Code Interpreter for Python execution
8. ✅ Working with existing agents
9. ✅ Existing conversation management
10. ✅ Explicit configuration settings
11. ✅ File search with vector stores
12. ✅ Hosted MCP tool integration
13. ✅ Image generation
14. ✅ Structured outputs with response format
15. ✅ Search context - Agentic mode
16. ✅ Search context - Semantic mode
17. ✅ Thread management for conversations
18. ✅ Web search capabilities

Each example showcases different capabilities and integration patterns available with Microsoft Agent Framework and Azure AI.